In [ ]:
import os, sys
import traceback
import pandas as pd
import re
import math
import numpy as np
from string import ascii_uppercase
import glob
import shutil

def Fluent_picklist(Filename, df_picklist, subfolder):
    
    df_picklist['Source_plate'] = '300mlSBSWater'
    # the names of Source_plate and Dest_plate will change with different plate types
    Dest_plate, Filename = IDT_plate_definition(Filename, df_picklist)

    df_picklist['Dest_plate'] = Dest_plate
        
    cols = ['Source_plate', 'Source Well', 'Dest_plate', 'Destination Well', 'Volume' ]
    
    df_picklist = df_picklist[cols]
    
    write_to_subfolder(Filename, df_picklist, subfolder)
    CSV_to_GWL_add_water(Filename, df_picklist, 'F;\n')
    # second mode W;\n will change tips after each transfer





def CSV_to_GWL_add_water(Filename, Fluent_picklist, transfer_mode):
    
    Step_list =[]

    for index, row in Fluent_picklist.iterrows():

        Volume = Fluent_picklist['Volume'].loc[index]

        S_plate = Fluent_picklist['Source_plate'].loc[index]
        S_well = Fluent_picklist['Source Well'].loc[index]
        Step_1 = str('A;'+ S_plate+';;;'+str(S_well)+';;'+str(Volume)+';;;;\n')
        Step_list.append(Step_1)

        D_plate = Fluent_picklist['Dest_plate'].loc[index]
        D_well = Fluent_picklist['Destination Well'].loc[index]
        Step_2 = str('D;'+ D_plate+';;;'+str(D_well)+';;'+str(Volume)+';;;;\n')
        Step_list.append(Step_2)
        
        Step_list.append(transfer_mode)

    Gwl_Filename = Filename[:-4]+'.gwl'
    
    F = open(Gwl_Filename, "w+")
    F.writelines(Step_list)
    F.close()



def Move_GWL_files_to_subfolder():
    
    source = os.getcwd()
    Dest_dir = os.path.join(source, 'gwl_worklists')
    print('\n Moving all gwl flies to:  \t', Dest_dir, '\n')

    if os.path.exists(Dest_dir):
        shutil.rmtree(Dest_dir)
    os.makedirs(Dest_dir)

    gwl_files = glob.glob('*.gwl')

    for file in gwl_files:
        file_name = os.path.join(source, file)
        shutil.move(file_name, Dest_dir)
        
    Plate_types['Echo Plate'] = ['EchoSource384']
    
    Plate_type_df = pd.DataFrame.from_dict(Plate_types, orient = 'index', columns = ['Value'])
    Plate_type_df = Plate_type_df.reset_index(drop = False)
    
    write_to_subfolder('Input_Plate_types.csv', Plate_type_df, 'gwl_worklists')
    print(Plate_type_df.to_string())